In [39]:
# import necessary packages
import numpy as np
import scipy as sci
import matplotlib as plt

In [40]:
class QuantumRegister:
    def __init__(self, N):
        # Initialize column vector
        self.N = N
        self.psi = 1 / np.sqrt(2**self.N) * np.zeros((2**N, 1), dtype=complex)
        # Set the amplitude of the first state
        self.psi[0] = 1
    
    def get_register(self):
        # Method to return the quantum state
        return self.psi
    
    def get_probabilities(self):
        return np.abs(self.psi)**2

    def measure_state(self):
        probabilities = np.abs(self.psi)**2
        # select a basis state randomly according to the probabilities
        measurement = np.random.choice(np.arange(len(self.psi)), p=probabilities.ravel())
        return "| " + np.binary_repr(measurement, width = self.N) + " >"

    def target_state(self):
        target = [1] * (self.N - 1)
        target.append(0)
        targetPrint = ''.join(str(x) for x in target)
        return "| " + targetPrint + " >"

    def ApplyGate(self,gate):
        self.psi = np.matmul(gate,self.psi)

    

In [41]:
class Hadamard:   
    @staticmethod
    def HadamardTransform(N):
        # define the initial Hadamard matrix
        hadamard_initial = 1 / np.sqrt(2) * np.array([[1, 1], [1, -1]])

        # number of iterations for the transformation
        num_iterations = N - 1

        # initialize the resulting tensor product matrix
        hadamard_result = hadamard_initial

        # perform tensor product iteratively
        for i in range(num_iterations):
            hadamard_result = np.kron(hadamard_initial, hadamard_result)

        return hadamard_result
    
    @staticmethod
    def ApplyHadamard(QuantReg):
        N = QuantReg.N
        HadamardGate = Hadamard.HadamardTransform(N)
        QuantReg.ApplyGate(HadamardGate)

In [42]:
class Oracle:
    @staticmethod
    def OracleGate(N,target):
        #initialise oracle matric with target
        oracleInitalise = np.eye(2**N)
        oracleInitalise[target -1][target-1]=-1
        
        return oracleInitalise
    
    @staticmethod
    def ApplyOracle(QuantReg):
        N = QuantReg.N
        target = (2**N) - 1
        OracleOperator = Oracle.OracleGate(N,target)
        QuantReg.ApplyGate(OracleOperator)

In [43]:
class Diffusion:
    @staticmethod
    def DiffusionOperator(N):
        # Define superposition state
        s = np.zeros((2**N, 1), dtype=complex) #/ np.sqrt(2**N)
        s[0][0]=1
        # Compute |s><s|
        ss_dagger = np.dot(s,s.T.conj())
        # ID Matric
        identity = np.eye(2**N)
        # Diffusion Operator
        diffusion =  (2 * ss_dagger) - identity 
        return diffusion
    
    @staticmethod
    def ApplyDiffusion(QuantReg):
        N = QuantReg.N
        DiffusionGate = Diffusion.DiffusionOperator(N)
        QuantReg.ApplyGate(DiffusionGate)

In [44]:
class Grover:
    @staticmethod
    def ApplyAlgorithm(QuantReg):
        N = QuantReg.N
        IdealLoop = ((np.pi/4) * np.sqrt(2**N)).astype(int)
        Hadamard.ApplyHadamard(QuantReg)
        for i in range(IdealLoop):
            Oracle.ApplyOracle(QuantReg)
            Hadamard.ApplyHadamard(QuantReg)
            Diffusion.ApplyDiffusion(QuantReg)
            Hadamard.ApplyHadamard(QuantReg)
        

In [45]:
"""
TESTING BLOCK
"""

"""creates the quantum register """
N=3
target = 7
quantum_Register1 = QuantumRegister(N)
print(quantum_Register1.get_register())
print("Current State: ",quantum_Register1.measure_state())
print("Target State:  ",quantum_Register1.target_state())

"""operating on the register using the gates"""
#Applying the Hadamard
Hcheck = Hadamard.HadamardTransform(N)
#print(Hcheck)

#Applying the Diffusion Operator
diffuse =  Diffusion.DiffusionOperator(N)
#print(diffuse)

#Applying the Oracle
Ocheck = Oracle.OracleGate(N,target)
#print(Ocheck)

"""Trying to work Grovers"""
Hadamard.ApplyHadamard(quantum_Register1)
for i in range(2):
    Oracle.ApplyOracle(quantum_Register1)
    Hadamard.ApplyHadamard(quantum_Register1)
    Diffusion.ApplyDiffusion(quantum_Register1)
    Hadamard.ApplyHadamard(quantum_Register1)
print(quantum_Register1.get_probabilities())
print(quantum_Register1.measure_state())


[[1.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]
 [0.+0.j]]
Current State:  | 000 >
Target State:   | 110 >
[[0.0078125]
 [0.0078125]
 [0.0078125]
 [0.0078125]
 [0.0078125]
 [0.0078125]
 [0.9453125]
 [0.0078125]]
| 110 >


In [46]:
qr = QuantumRegister(3)
print("Begin State: ", qr.measure_state())
print(qr.get_probabilities())


Grover.ApplyAlgorithm(qr)
print("After Algorithm: ", qr.measure_state())
print(qr.get_probabilities())

Begin State:  | 000 >
[[1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]
After Algorithm:  | 110 >
[[0.0078125]
 [0.0078125]
 [0.0078125]
 [0.0078125]
 [0.0078125]
 [0.0078125]
 [0.9453125]
 [0.0078125]]


In [47]:
qrBig = QuantumRegister(3)
print("Begin State: ", qrBig.measure_state())
print(qrBig.get_probabilities())


Grover.ApplyAlgorithm(qrBig)
print("After Algorithm: ", qrBig.measure_state())
print(qrBig.get_probabilities())

Begin State:  | 000 >
[[1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]
After Algorithm:  | 110 >
[[0.0078125]
 [0.0078125]
 [0.0078125]
 [0.0078125]
 [0.0078125]
 [0.0078125]
 [0.9453125]
 [0.0078125]]


In [48]:
import numpy as np
from numpy.random import randint
from numpy.linalg import matrix_power

# Function to calculate greatest common divisor (gcd) 
def gcd(a, b):
    while b != 0:
        a, b = b, a % b
    return a

# Continued fraction expansion 
def continued_fraction(a, N):
    r = a / N
    frac = [0, int(r)]
    while frac[-1] != r:
        r = 1 / (r - int(r))
        frac.append(int(r))
    return frac

# Find period using continued fraction expansion
def find_period(a, N):
    frac = continued_fraction(a, N)
    if len(frac) % 2 == 0:
        frac.pop()
    return frac

# Perform quantum phase estimation
def quantum_phase_estimation(U, n):
    # Create Hadamard gate
    H = 1 / np.sqrt(2) * np.array([[1, 1], [1, -1]])

    # Initialize qubits
    qubits = np.zeros((2**n, 1))
    qubits[0] = 1

    # Apply Hadamard gate to first register
    for i in range(n):
        qubits = np.kron(H, np.eye(2**(n-1))) @ qubits

    # Apply controlled-U gates
    for _ in range(int(2**(n-1))):
        qubits = np.kron(np.eye(2), U) @ qubits

    # Apply inverse quantum Fourier transform
    qubits = np.fft.ifft(qubits, axis=0)

    # Measure and return result
    return qubits.argmax()

# Perform Shor's algorithm
def shors_algorithm(N, attempts=10):
    for _ in range(attempts):
        a = randint(2, N)
        if gcd(a, N) != 1:
            return gcd(a, N)
        else:
            frac = find_period(a, N)
            for period in frac[1:]:
                if period % 2 == 0:
                    r = int(matrix_power(a, period/2))
                    if (r + 1) % N != 0:
                        p = gcd(r + 1, N)
                        return p
    return "Failed to find factors"

In [51]:
shorsTests = shors_algorithm(123450)
print("factors: ",shorsTests)

factors:  75
